In [56]:
# Following this repo example: https://github.com/Azure-Samples/azure-search-python-samples/blob/main/Quickstart/v11/azure-search-quickstart.ipynb
# Index - https://portal.azure.com/#view/Microsoft_Azure_Search/Index.ReactView/id/%2Fsubscriptions%2F2f8a2563-4e90-4a88-8e6c-56fa0f12cb7e%2FresourceGroups%2FSmart-search%2Fproviders%2FMicrosoft.Search%2FsearchServices%2Fsmart-seach-test1%23azureblob-index/location/East%20US/sku/standard
# Uncomment the next line if you need to install the azure-search-documents client library:
!pip install azure-search-documents --pre


In [58]:

# This sample uses version: 11.1.0
!pip show azure-search-documents

import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

Name: azure-search-documents
Version: 11.4.0b3
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: isodate, azure-common, azure-core
Required-by: 


In [60]:
# Set the service endpoint and API key from the environment

service_name = "smart-search-test1" #"YOUR-SEARCH-SERVICE-NAME"
admin_key = "H7XVHXwrslvcD7TSvcC2PuJqJONRCwPYzPJjeVCvZbAzSeBAR1di" #"YOUR-SEARCH-SERVICE-ADMIN-API-KEY"

index_name = "azureblob-index"

# Create an SDK client
# endpoint = "https://{}.search.windows.net/".format(service_name)
endpoint = "https://smart-seach-test1.search.windows.net"
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))


In [61]:
# Delete the index if it exists
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Index azureblob-index Deleted


In [82]:
# Specify the index schema
name = index_name
fields = [
        SearchableField(name="pmid", type=SearchFieldDataType.String, filterable=True, facetable=True, key=True),
        SearchableField(name="journal", type=SearchFieldDataType.String, filterable=True, facetable=True, sortable=True),
        SearchableField(name="title", type=SearchFieldDataType.String, filterable=True, facetable=True, sortable=True),
        SearchableField(name="abstract", type=SearchFieldDataType.String, filterable=True, facetable=True, sortable=True),
        
        SearchableField(name="keywords", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=False),
        
        SearchableField(name="pub_type", type=SearchFieldDataType.String, filterable=True, facetable=True, sortable=True),
        SearchableField(name="authors", type=SearchFieldDataType.String, filterable=True, facetable=True, sortable=True),
        SearchableField(name="doi", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        SearchableField(name="label", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
    
    ]
cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []

In [83]:
# semantic_config = {
#     "name": "semantic",
#     "searchMode": "all",
#     "sourceFields": ["title", "abstract"]
# }
# search_config = {"searchFields": {"searchableFields": index_name}, "queryType": [semantic_config]}


In [78]:
index = SearchIndex(
    name=name,
    fields=fields,
    scoring_profiles=scoring_profiles,
    #suggesters = suggester,
    cors_options=cors_options,
    )

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

(ResourceNameAlreadyInUse) Cannot create index 'azureblob-index' because it already exists.
Code: ResourceNameAlreadyInUse
Message: Cannot create index 'azureblob-index' because it already exists.
Exception Details:	(CannotCreateExistingIndex) Cannot create index 'azureblob-index' because it already exists.
	Code: CannotCreateExistingIndex
	Message: Cannot create index 'azureblob-index' because it already exists.


In [79]:

documents = [
    {
    "@search.action": "upload",
    "pmid": "32519164",
    "journal": "J Thromb Thrombolysis",
    "title": "Potential role for tissue factor in the pathogenesis of hypercoagulability associated with in COVID-19.",
    "abstract": "In December 2019, a new and highly contagious infectious disease emerged in Wuhan, China. The etiologic agent was identified as a novel coronavirus, now known as Severe Acute Syndrome Coronavirus-2 (SARS-CoV-2). Recent research has revealed that virus entry takes place upon the union of the virus S surface protein with the type I transmembrane metallo-carboxypeptidase, angiotensin converting enzyme 2 (ACE-2) identified on epithelial cells of the host respiratory tract. Virus triggers the synthesis and release of pro-inflammatory cytokines, including IL-6 and TNF-alpha and also promotes downregulation of ACE-2, which promotes a concomitant increase in levels of angiotensin II (AT-II). Both TNF-alpha and AT-II have been implicated in promoting overexpression of tissue factor (TF) in platelets and macrophages. Additionally, the generation of antiphospholipid antibodies associated with COVID-19 may also promote an increase in TF. TF may be a critical mediator associated with the development of thrombotic phenomena in COVID-19, and should be a target for future study.",
    "keywords": ["covid-19","il-6","sars-cov-2","tnf-alpha","thrombosis","tissue factor"],
    "pub_type": "Journal Article;Review",
    "authors": "Bautista-Vargas, Mario;Bonilla-Abadia, Fabio;Canas, Carlos A",
    "doi": "10.1007/s11239-020-02172-x",
    "label": "Treatment;Mechanism",
    },
    {
    "@search.action": "upload",
    "pmid": "34211701",
    "journal": "F1000Res",
    "title": "Risk factors for mortality in hospitalized patients with COVID-19 from three hospitals in Peru: a retrospective cohort study.",
    "abstract": "BACKGROUND: Peru was one of the countries with the highest COVID-19 mortality worldwide during the first stage of the pandemic. It is then relevant to evaluate the risk factors for mortality in patients hospitalized for COVID-19 in three hospitals in Peru in 2020, from March to May, 2020. Methods: We carried out a retrospective cohort study. The population consisted of patients from three Peruvian hospitals hospitalized for a diagnosis of COVID-19 during the March-May 2020 period. Independent sociodemographic variables, medical history, symptoms, vital functions, laboratory parameters and medical treatment were evaluated. In-hospital mortality was assessed as the outcome. We performed Cox regression models (crude and adjusted) to evaluate risk factors for in-hospital mortality. Hazard ratios (HR) with their respective 95% confidence intervals (95% CI) were calculated. Results: We analyzed 493 hospitalized adults; 72.8% (n=359) were male and the mean age was 63.3 +/- 14.4 years. COVID-19 symptoms appeared on average 7.9 +/- 4.0 days before admission to the hospital, and the mean oxygen saturation on admission was 82.6 +/- 13.8. While 67.6% (n=333) required intensive care unit admission, only 3.3% (n=16) were admitted to this unit, and 60.2% (n=297) of the sample died. In the adjusted regression analysis, it was found that being 60 years old or older (HR=1.57; 95% CI: 1.14-2.15), having two or more comorbidities (HR=1.53; 95% CI: 1.10-2.14), oxygen saturation between 85-80% (HR=2.52; 95% CI: 1.58-4.02), less than 80% (HR=4.59; 95% CI: 3.01-7.00), and being in the middle (HR=1.65; 95% CI: 1.15-2.39) and higher tertile (HR=2.18; 95% CI: 1.51-3.15) of the neutrophil-to-lymphocyte ratio, increased the risk of mortality. Conclusions: The risk factors found agree with what has been described in the literature and allow the identification of vulnerable groups in whom monitoring and early identification of symptoms should be prioritized in order to reduce mortality.",
    "keywords": ["adults","covid-19","latin america.","mortality","sars-cov-2"],
    "pub_type": "Journal Article;Research Support, Non-U.S. Gov't",
    "authors": "Diaz-Velez, Cristian;Urrunaga-Pastor, Diego;Romero-Cerdan, Anthony;Pena-Sanchez, Eric Ricardo;Fernandez Mogollon, Jorge Luis;Cossio Chafloque, Julio Darwin;Marreros Ascoy, Gaston Cristobal;Benites-Zapata, Vicente A",
    "doi": "10.12688/f1000research.51474.1",
    "label": "Diagnosis;Treatment",
    },
        {
    "@search.action": "upload",
    "pmid": "34367415",
    "journal": "Int J Clin Exp Pathol",
    "title": "Inflammation in COVID-19: from pathogenesis to treatment.",
    "abstract": "The coronavirus disease 2019 (COVID-19) was declared a pandemic in March 2020 by the World Health Organization (WHO). To date, there were > 163 million confirmed cases of COVID-19 and the disease has claimed > 3.3 million lives globally. As with many other diseases, inflammation is a key feature of COVID-19. When inflammation is overwhelming, it may lead to unfavorable outcomes or even death. Scientists all over the world are working tirelessly in search of therapeutic strategies to suppress or modulate inflammation in COVID-19. This review gives an overview of the role of inflammation in COVID-19. It also critically examines the various treatment approaches that target the immune system and inflammation in COVID-19, as well as highlights the key findings in the numerous studies conducted thus far.",
    "keywords": ["covid-19","sars-cov-2","cytokine storm","inflammation","pharmacotherapy"],
    "pub_type": "Journal Article;Review",
    "authors": "Wong, Rebecca S Y",
    "doi": "",
    "label": "Mechanism;Treatment",
    },
    {
    "@search.action": "upload",
    "pmid": "32691006",
    "journal": "J Tradit Complement Med",
    "title": "Dietary therapy and herbal medicine for COVID-19 prevention: A review and perspective.",
    "abstract": "A novel coronavirus disease (COVID-19), transmitted from humans to humans, has rapidly become the pandemic responsible for the current global health crisis. COVID-19 is caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which is said to be of zoonotic origin. This review describes the etiology and signs and symptoms as well as the current allopathic therapy for COVID-19. Additionally, findings of previous studies on the immunomodulatory effects and antiviral activities of particular foods and herbs on influenza virus and coronaviruses have been collated, with the aim of promoting the use of dietary therapy and herbal medicine as COVID-19 preventive therapies, while specific drugs and vaccines are yet to be discovered or are still under development. The volume of existing reports is irrefutable evidence that foods and herbs possess a potential antiviral ability against SARS-CoV-2 and can prevent COVID-19. Foods and herbs could be used as dietary or complementary therapy to prevent infection and strengthen immunity, as antiviral agents for masks, as disinfectants to curb aerosol transmission, or as sanitizing agents to disinfect surfaces. However, these hypotheses need to be experimentally verified for SARS-CoV-2 and COVID-19 patients.",
    "keywords": ["covid-19","coronavirus","dietary therapy","herbal medicine","herbs","prevention","sars-cov-2"],
    "pub_type": "Journal Article;Review",
    "authors": "Panyod, Suraphan;Ho, Chi-Tang;Sheen, Lee-Yan",
    "doi": "10.1016/j.jtcme.2020.05.004",
    "label": "Treatment;Prevention",
    }
]

In [85]:
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

Upload of new document succeeded: True


In [87]:
# Specify a valid semantic configuration for your query
semantic_config = {
    "name": "default",
    "queryAlterationEnabled": True
}
#result.parameters["semanticConfiguration"] = semantic_config

In [93]:
results =  search_client.search(search_text="J", include_total_count=True, query_type="semantic", query_language="en-us")
# query.parameters["semanticConfiguration"] = semantic_config

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print("{}: {}".format(result["journal"], result["authors"]))

NameError: name 'query' is not defined

In [54]:
print(results)

<iterator object azure.core.paging.ItemPaged at 0x7f5ddd7edf40>


In [67]:
results =  search_client.search(search_text="covid-19", include_total_count=True, select='journal,abstract,keywords')

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print("{}: {}: {}".format(result["journal"], result["abstract"], result["keywords"]))

Total Documents Matching Query: 4
J Tradit Complement Med: A novel coronavirus disease (COVID-19), transmitted from humans to humans, has rapidly become the pandemic responsible for the current global health crisis. COVID-19 is caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), which is said to be of zoonotic origin. This review describes the etiology and signs and symptoms as well as the current allopathic therapy for COVID-19. Additionally, findings of previous studies on the immunomodulatory effects and antiviral activities of particular foods and herbs on influenza virus and coronaviruses have been collated, with the aim of promoting the use of dietary therapy and herbal medicine as COVID-19 preventive therapies, while specific drugs and vaccines are yet to be discovered or are still under development. The volume of existing reports is irrefutable evidence that foods and herbs possess a potential antiviral ability against SARS-CoV-2 and can prevent COVID-19. Fo

In [69]:
# results =  search_client.search(search_text="*", facets=["Category"])

# facets = results.get_facets()

# for facet in facets["Category"]:
#     print("    {}".format(facet))

In [70]:
result = search_client.get_document(key="3")

print("Details for journal '3' are:")
print("        Name: {}".format(result["journal"]))
print("      Author: {}".format(result["authors"]))
#print("    Category: {}".format(result["Category"]))

ResourceNotFoundError: Operation returned an invalid status 'Not Found'

In [ ]:
# FIX / UNDERSTAND SUGGESTER
search_suggestion = 'sa'
results = search_client.autocomplete(search_text=search_suggestion, suggester_name="sg", mode='twoTerms')

print("Autocomplete for:", search_suggestion)
for result in results:
    print (result['text'])

In [ ]:
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

In [ ]:
try:
    result = admin_client.get_index(index_name)
    print (result)
except Exception as ex:
    print (ex)